# Model Entailment 1

This is the first of several notebooks to explore logical entailments in SysML v2 models, as well as the means by which to encode them.

In [1]:
import json
import pymbe.api as pm

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import build_from_classifier_pattern, \
                                    new_element_ownership_pattern, \
                                    build_from_binary_relationship_pattern, \
                                    build_unioning_superset_classifier, \
                                    build_from_feature_pattern, \
                                    build_from_binary_connector_pattern

from pymbe.query.metamodel_navigator import is_type_undefined_mult, \
                                    is_multiplicity_one, \
                                    is_multiplicity_specific_finite, \
                                    get_finite_multiplicity_types, \
                                    identify_connectors_one_side, \
                                    get_lower_multiplicty, \
                                    get_upper_multiplicty

from uuid import uuid4

## Load up and explore basic model

Load up a basic model in order to have basic package and namespace into which to add additional elements.

In [2]:
filename = "Model_Loader_Test_Level_2"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()).parent.parent / "tests/fixtures" / filename

level2 = pm.Model.load_from_post_file(json_file)
level2

<SysML v2 Model (C:\Users\Bjorn Cole\Documents\GitHub\pyMBE\tests\fixtures\Model_Loader_Test_Level_2.json)>

In [3]:
len(level2.elements)

47

In [4]:
classifiers = [ele for ele in level2.elements.values() if ele._metatype == 'Classifier']
classifiers

[Context «Classifier»]

Locate the root package of the model to which new elements will be added.

In [5]:
base_package = classifiers[0].owningRelationship.owningRelatedElement
base_package

Model Loader Test Level 2 «Package»

## Example Application of Reasoning 1 - Build Common Class from Examples

In this example, we look at a series of specific examples of an item, 4 bicycle wheels.

First, add these elements to the model.

In [6]:
bw1 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #1", model=level2, specific_fields={}, metatype="Classifier")
bw2 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #2", model=level2, specific_fields={}, metatype="Classifier")
bw3 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #3", model=level2, specific_fields={}, metatype="Classifier")
bw4 = build_from_classifier_pattern(owner=base_package, name="Bike Wheel #4", model=level2, specific_fields={}, metatype="Classifier")

In [7]:
bw1.owningRelationship

<OwningMembership([Model Loader Test Level 2 «Package»] ←→ [Bike Wheel #1 «Classifier»])>

Now, create a new superclass that all of these specific wheels can be grouped into.

In [8]:
new_wheel = build_unioning_superset_classifier(classes=[bw1, bw2, bw3, bw4],
                                    super_name="Bike Wheel",
                                    model=level2,
                                    owner=base_package,
                                    added_fields={})

Check that the wheel has its specific versions.

In [9]:
new_wheel.reverseSubclassification

[Bike Wheel #1 «Classifier»,
 Bike Wheel #2 «Classifier»,
 Bike Wheel #3 «Classifier»,
 Bike Wheel #4 «Classifier»]

In [10]:
base_package.ownedMember

[Context «Classifier»,
 Bike Wheel #1 «Classifier»,
 Bike Wheel #2 «Classifier»,
 Bike Wheel #3 «Classifier»,
 Bike Wheel #4 «Classifier»,
 Bike Wheel «Classifier»]

## Developing unrolling rule around connectors

Rule 1 - find connectors with ends that have a multiplicity of 1 and then specialize them.

In [11]:
connectors = [ele for ele in level2.elements.values() if ele._metatype == 'Connector']

In [12]:
efms = [ele for ele in level2.elements.values() if ele._metatype == 'EndFeatureMembership']
efms

[<EndFeatureMembership([<BindingConnector([4263296a-9ba6-4ebf-96a3-aff897ae1387 «Feature»] ←→ [972c4cc3-c930-49d4-8468-05a183106ce5 «Feature»])>] ←→ [fd93f251-98e5-4c9f-98c3-ab112690a971 «Feature»])>,
 <EndFeatureMembership([<BindingConnector([4263296a-9ba6-4ebf-96a3-aff897ae1387 «Feature»] ←→ [972c4cc3-c930-49d4-8468-05a183106ce5 «Feature»])>] ←→ [5e047bcc-a32e-4005-af3a-cb279d6f9f1b «Feature»])>,
 <EndFeatureMembership([<Connector([Side 1 «Feature»] ←→ [Side 2 «Feature»])>] ←→ [007d12b8-fc9d-4ad9-8b35-7dd35900aca5 «Feature»])>,
 <EndFeatureMembership([<Connector([Side 1 «Feature»] ←→ [Side 2 «Feature»])>] ←→ [911ec4fb-ec82-499f-8f7f-b2e9a39288ad «Feature»])>]

In [13]:
for end_feature in connectors[0].throughEndFeatureMembership:
    print(end_feature)
    if 'throughReferenceSubsetting' in end_feature._derived:
        print(f"Feature references {end_feature.throughReferenceSubsetting[0]}")

007d12b8-fc9d-4ad9-8b35-7dd35900aca5 «Feature»
Feature references Side 1 «Feature»
911ec4fb-ec82-499f-8f7f-b2e9a39288ad «Feature»
Feature references Side 2 «Feature»


In [14]:
features = [ele for ele in level2.elements.values() if ele._metatype == 'Feature']
features

[Value «Feature»,
 Side 1 «Feature»,
 out result «Feature»,
 Value «Feature»,
 Side 2 «Feature»,
 4263296a-9ba6-4ebf-96a3-aff897ae1387 «Feature»,
 972c4cc3-c930-49d4-8468-05a183106ce5 «Feature»,
 fd93f251-98e5-4c9f-98c3-ab112690a971 «Feature»,
 5e047bcc-a32e-4005-af3a-cb279d6f9f1b «Feature»,
 007d12b8-fc9d-4ad9-8b35-7dd35900aca5 «Feature»,
 911ec4fb-ec82-499f-8f7f-b2e9a39288ad «Feature»]

Look for multiplicity ranges in the model that are set to 1.

In [15]:
[is_type_undefined_mult(ft) for ft in features]

[True, True, True, True, False, True, True, True, True, True, True]

In [16]:
[is_multiplicity_one(ft) for ft in features]

[False, False, False, False, True, False, False, False, False, False, False]

In [17]:
[is_multiplicity_specific_finite(ft) for ft in features]

[False, False, False, False, False, False, False, False, False, False, False]

Find all types (classifiers and features) that have a declared multiplicity that is finite.

In [18]:
finite_mult = get_finite_multiplicity_types(level2)
finite_mult

[Side 2 «Feature»]

In [19]:
get_lower_multiplicty(finite_mult[0])

1

In [20]:
get_upper_multiplicty(finite_mult[0])

1

Find where ends the connection is bound to other features in the model.

In [21]:
refsubs = [ele for ele in level2.elements.values() if ele._metatype == 'ReferenceSubsetting']
refsubs

[<ReferenceSubsetting([fd93f251-98e5-4c9f-98c3-ab112690a971 «Feature»] ←→ [4263296a-9ba6-4ebf-96a3-aff897ae1387 «Feature»])>,
 <ReferenceSubsetting([5e047bcc-a32e-4005-af3a-cb279d6f9f1b «Feature»] ←→ [972c4cc3-c930-49d4-8468-05a183106ce5 «Feature»])>,
 <ReferenceSubsetting([007d12b8-fc9d-4ad9-8b35-7dd35900aca5 «Feature»] ←→ [Side 1 «Feature»])>,
 <ReferenceSubsetting([911ec4fb-ec82-499f-8f7f-b2e9a39288ad «Feature»] ←→ [Side 2 «Feature»])>]

In [22]:
conns_to_cover = identify_connectors_one_side(connectors)
conns_to_cover

[<Connector([Side 1 «Feature»] ←→ [Side 2 «Feature»])>]

In [23]:
conns_to_cover[0].source

[Side 1 «Feature»]

The function below shows the creation of a connector which requires many elements to be created (the connection itself, the ends, references out to other Features in the mind, specialization relationship.

In [24]:
test_conn = build_from_binary_connector_pattern(
    name="Test Connector",
    source_role_name="source end",
    target_role_name="target end",
    source=bw1,
    target=bw2,
    model=level2,
    metatype="Connector",
    owner=base_package,
    specific_fields={}
)

In [25]:
test_conn._data

{'owningRelationship': {'@id': '936c1bbb-80f7-4c80-b1d7-d59da93c585e'},
 'owningRelatedElement': None,
 'source': [{'@id': '4416e243-936f-4fa4-8d9a-1ed39f060db9'}],
 'ownedRelationship': [{'@id': '0f99a9ce-a536-4205-98a8-dc10e15df295'},
  {'@id': 'e31c14e6-fc12-40e5-ab81-afbed5b76a4f'}],
 'target': [{'@id': '77300e29-24cc-44e9-b1f3-03bd738d83c5'}],
 'ownedRelatedElement': [],
 'isOrdered': False,
 'aliasIds': [],
 'isEnd': False,
 'direction': [],
 'isSufficient': False,
 'isReadOnly': False,
 'isImpliedIncluded': False,
 'elementId': '',
 'isAbstract': False,
 'declaredShortName': '',
 'isImplied': False,
 'isUnique': False,
 'isPortion': False,
 'isDirected': False,
 'isComposite': False,
 'isDerived': False,
 'declaredName': 'Test Connector',
 'relatedFeature': None,
 'documentation': None,
 'connectorEnd': None,
 'owningMembership': None,
 'ownedIntersecting': None,
 'chainingFeature': None,
 'ownedDisjoining': None,
 'ownedDifferencing': None,
 'ownedSpecialization': None,
 'ownin

Get the root package again for the new connections to be owned by.

In [26]:
top_elements = [ele for ele in level2.ownedElement if ele._metatype == "Namespace"][0].throughOwningMembership
top_package = [ele for ele in top_elements if ele._metatype == "Package"][0]
top_package

Model Loader Test Level 2 «Package»

In [27]:
connectors = [ele for ele in level2.elements.values() if ele._metatype == 'Connector']
connectors

[<Connector([Side 1 «Feature»] ←→ [Side 2 «Feature»])>,
 <Connector([Bike Wheel #1 «Classifier»] ←→ [Bike Wheel #2 «Classifier»])>]

Look at end feature memberships and the links between the classifiers connected.

In [28]:
connectors[1].ownedRelationship

[<EndFeatureMembership([<Connector([Bike Wheel #1 «Classifier»] ←→ [Bike Wheel #2 «Classifier»])>] ←→ [source end «Feature»])>,
 <EndFeatureMembership([<Connector([Bike Wheel #1 «Classifier»] ←→ [Bike Wheel #2 «Classifier»])>] ←→ [target end «Feature»])>]

In [29]:
connectors[1].ownedRelationship[0].target[0].throughReferenceSubsetting

[Bike Wheel #1 «Classifier»]

In [30]:
connectors[1].ownedRelationship[1].target[0].throughReferenceSubsetting

[Bike Wheel #2 «Classifier»]